In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'/home/ehsan/PycharmProjects/Chicken-Disease-Classification/Chicken-Disease-Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class CallbacksConfig:
    root_dir: Path
    tensorboard_log_dir: Path
    checkpoint_dir: Path
    

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_dirs

In [15]:
class ConfigurationManager:
    def __init__(
            self,
            config_file=CONFIG_FILE_PATH,
            params_file=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file)
        self.params_file = read_yaml(params_file)
        
        create_dirs([self.config.artifacts_root])
        
    def get_callbacks_config(self) -> CallbacksConfig:
        config1 = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config1.checkpoint_model_file)
        create_dirs([
            model_ckpt_dir,
            config1.tensorboard_log_dir
                     ])
        callbacks_config1 = CallbacksConfig(
            root_dir=Path(config1.root_dir),
            tensorboard_log_dir=Path(config1.tensorboard_log_dir),
            checkpoint_dir=Path(config1.checkpoint_model_file)
        )
        return callbacks_config1

In [8]:
import time
import tensorflow as tf

2024-02-22 12:03:12.666989: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-02-22 12:03:17,887 - INFO - utils - NumExpr defaulting to 8 threads.


In [10]:
class Callbacks:
    def __init__(self, config: CallbacksConfig):
        self.config = config
        
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d--%H:%M:%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_log_dir , f'tb_log_{timestamp}')
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_dir,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks,
                self._create_ckpt_callbacks]

In [16]:
try :
    config = ConfigurationManager()
    callbacks_config = config.get_callbacks_config()
    callbacks = Callbacks(callbacks_config)
    callbacks_list = callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

2024-02-22 12:14:55,794 - INFO - common - Loaded yaml file config/config.yaml
2024-02-22 12:14:55,802 - INFO - common - Loaded yaml file params.yaml
2024-02-22 12:14:55,805 - INFO - common - created directory artifacts
2024-02-22 12:14:55,807 - INFO - common - created directory artifacts/prepare_callbacks/checkpoint_dir
2024-02-22 12:14:55,808 - INFO - common - created directory artifacts/prepare_callbacks/tensorboard_log_dir
